In [ ]:
# 首先确定数据集

In [1]:
import cv2
import numpy as np


def detect_license_plate(img):
    """
    识别车牌
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 100, 200)

    contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for i in range(len(contours)):
        cnt = contours[i]
        area = cv2.contourArea(cnt)

        if area < 2000 or area > 10000:
            continue

        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        width, height = rect[1]
        if height > width:
            width, height = height, width

        ratio = height / width
        if ratio < 2 or ratio > 5:
            continue

        cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

    cv2.imshow("License Plate Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == '__main__':
    img = cv2.imread("car1.jpg")
    detect_license_plate(img)


In [10]:
import cv2
import numpy as np
import tensorflow as tf
model = tf.keras.models.load_model(
    "D:\code\python\machine_learning\cnn_zuo_tu.h5")
# 读取图片
img = cv2.imread('car1.jpg')

# YOLOv3目标检测
net = cv2.dnn.readNet("D:\code\python\machine_learning\yolo\yolov3.weights",
                      "D:\code\python\machine_learning\yolo\yolov3.cfg")
classes = []
with open("D:\code\python\machine_learning\yolo\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

height, width, channels = img.shape

# 对图像进行预处理，包括缩放和二值化
blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(
    416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# 找出车牌区域
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # 对车牌区域进行裁剪
        plate = img[y:y+h, x:x+w]

        # 对裁剪后的车牌区域进行字符分割
        gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(
            thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        char_list = []
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            ratio = h/w
            if ratio > 1.5 and ratio < 3 and h/plate.shape[0] > 0.3:
                cv2.rectangle(plate, (x, y), (x + w, y + h), (0, 255, 0), 2)
                roi = thresh[y:y+h, x:x+w]
                roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
                roi = cv2.dilate(roi, (3, 3))
                char_list.append((x, roi))

        # 按字符位置从左到右排序
        char_list = sorted(char_list, key=lambda x: x[0])

        # 识别字符并输出结果
        text = ''
        for char in char_list:
            x, roi = char
            roi = np.array(roi, dtype=np.float32)
            roi /= 255
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=3)
            result = model.predict(roi)
            result_number = np.argmax(result)
            text += str(result_number)

        print("车牌号码：", text)
        cv2.imshow("plate", plate)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np

# 读取图片
img = cv2.imread('car.jpg')

# YOLOv3目标检测
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

height, width, channels = img.shape

# 对图像进行预处理，包括缩放和二值化
blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# 找出车牌区域
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 对车牌区域进行裁剪
plate = img[y:y+h, x:x+w]

# 对裁剪后的车牌区域进行字符分割
gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

...

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


In [10]:
#好的，下面是车牌字符分割的Python代码示例。

import cv2

# 读取裁剪后的车牌图像
plate_img = cv2.imread('car1.jpg')

# 灰度化和二值化处理
gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# 查找车牌字符的轮廓，并按照从左到右的顺序进行排序
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

# 将每个字符的图像分别保存，并显示出来
for i, contour in enumerate(contours):
    # 获得每个字符的坐标、宽度和高度
    x, y, w, h = cv2.boundingRect(contour)
    # 根据坐标和大小截取出每个字符的图像
    char_img = plate_img[y:y+h, x:x+w]
    # 将截取出来的字符图像保存为文件，文件名为字符的序号
    cv2.imwrite(str(i) + '.jpg', char_img)
    # 在原图上用矩形框标识出每个字符的位置
    cv2.rectangle(plate_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # 在窗口中显示每个字符的图像
    cv2.imshow('Char '+str(i), char_img)

# 在窗口中显示标识出字符位置的原图
cv2.imshow('Plate', plate_img)

# 等待用户按下任意键后退出程序
cv2.waitKey(0)
cv2.destroyAllWindows()


#上面的代码首先读取了车牌图像，并将其转换为灰度图像，然后使用 OTSU 自适应二值化，将车牌字符变为白色，背景变为黑色。接着，使用 `cv2.findContours` 函数查找车牌字符的轮廓，按照从左到右的顺序进行排序，然后循环处理每个字符，截取出它的图像，保存为文件，并在原图上用矩形框标识出每个字符的位置。最后，在窗口中分别显示每个字符的图像和标识出字符位置的原图。

In [9]:
# 好的，以下是基于 OpenCV 的 Python 代码示例：


import cv2
import numpy as np

# 读取图像
img = cv2.imread('car.jpg')

# 转换颜色空间
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 创建颜色区间
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])
lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])
lower_yellow = np.array([20, 50, 50])
upper_yellow = np.array([30, 255, 255])

# 提取车牌颜色
mask_red = cv2.inRange(hsv, lower_red, upper_red)
mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

# 统计颜色像素数量
red_pixels = cv2.countNonZero(mask_red)
blue_pixels = cv2.countNonZero(mask_blue)
yellow_pixels = cv2.countNonZero(mask_yellow)

# 计算颜色概率分布
total_pixels = img.shape[0] * img.shape[1]
red_prob = red_pixels / total_pixels
blue_prob = blue_pixels / total_pixels
yellow_prob = yellow_pixels / total_pixels

# 输出颜色概率分布
print("Red probability:", red_prob)
print("Blue probability:", blue_prob)
print("Yellow probability:", yellow_prob)

# 显示图像和颜色区域
cv2.imshow('Image', img)
cv2.imshow('Red', mask_red)
cv2.imshow('Blue', mask_blue)
cv2.imshow('Yellow', mask_yellow)
cv2.waitKey(0)
cv2.destroyAllWindows()


# 在本示例中，我们使用 `cv2.inRange()` 函数提取颜色区间内的像素，并使用 `cv2.countNonZero()` 函数计算该颜色所占像素数量。最终，我们计算每个颜色概率分布，并输出结果。

# 请注意，颜色区间的值是在 HSV 颜色空间下定义的，因为它对于颜色识别更有效。你可以根据你的需求修改颜色区间值。在本示例中，我们提取了红色、蓝色和黄色车牌。

Red probability: 0.005276102824500159
Blue probability: 0.0432005712472231
Yellow probability: 0.09298635353855919


In [4]:
import cv2
import numpy as np
import pytesseract

# 设置识别语言为英文
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
custom_config = r'-l eng --oem 3 --psm 6'

def plate_detection(image):
    # 转换为灰度图像
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 提取水平和垂直方向上的图像边缘
    dx = cv2.Sobel(gray_img, cv2.CV_32F, 1, 0)
    dy = cv2.Sobel(gray_img, cv2.CV_32F, 0, 1)
    
    # 取绝对值
    dx = cv2.convertScaleAbs(dx)
    dy = cv2.convertScaleAbs(dy)
    
    # 结合水平方向和垂直方向的边缘
    edge_img = cv2.addWeighted(dx, 0.5, dy, 0.5, 0)
    
    # 使用二值化处理，生成二值图像
    _, threshold_img = cv2.threshold(edge_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 膨胀操作，提升连通性
    kernel = np.ones((3, 3), np.uint8)
    morph_img = cv2.morphologyEx(threshold_img, cv2.MORPH_DILATE, kernel)
    
    # 在图像中查找指定形状的轮廓，这里是一个矩形
    contours, hierarchy = cv2.findContours(morph_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 定位最大轮廓区域，即车牌所在的位置。因为矩形面积是宽*高，因此定位最大矩形区域即定位车牌位置
    max_area = 0
    plate_rect = None
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            plate_rect = cv2.boundingRect(contour)
    
    # 返回车牌位置矩形框
    return plate_rect

def segment_characters(image, plate_rect):
    # 裁剪出车牌区域
    x, y, w, h = plate_rect
    plate_image = image[y:y+h, x:x+w]
    
    # 将车牌进行腐蚀操作
    kernel = np.ones((3,3), np.uint8)
    eroded_plate_image = cv2.erode(plate_image, kernel, iterations=1)
    
    # 将车牌转换为灰度图像
    gray_img = cv2.cvtColor(eroded_plate_image, cv2.COLOR_BGR2GRAY)
    
    # 使用二值化处理，生成二值图像
    _, threshold_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 使用开操作去除噪点
    kernel = np.ones((3,3), np.uint8)
    opening_img = cv2.morphologyEx(threshold_img, cv2.MORPH_OPEN, kernel)
    
    # 查找图像中轮廓
    contours, hierarchy = cv2.findContours(opening_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 裁剪并保存识别出的字符
    segments = []
    for contour in contours:
        contour_rect = cv2.boundingRect(contour)
        x1, y1, w1, h1 = contour_rect
        aspect_ratio = w1 / h1
        if 0.2 < aspect_ratio < 1.0:
            segment = opening_img[y1:y1+h1, x1:x1+w1]
            segments.append(segment)
    
    #返回字符片段
    return segments

def recognize_characters(segments):
    # 识别每个字符
    characters = []
    for segment in segments:
        character = pytesseract.image_to_string(segment, config=custom_config).strip()
        characters.append(character)
    
    # 返回所有字符
    return characters

def recognize_color(image):
    #将图像从BGR颜色空间转换到HSV颜色空间
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # 设置HSV颜色阈值上下限值
    lower_blue = np.array([100, 43, 46])
    upper_blue = np.array([124, 255, 255])
    
    # 筛选符合指定颜色范围的像素点
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # 计算符合条件像素点的数量，设定数量阈值以确定车牌颜色
    pixel_count = cv2.countNonZero(mask)
    
    # 如果符合条件像素点的数量大于设定阈值，则返回蓝色车牌，否则返回黄色车牌
    if pixel_count > 2000:
        return 'blue'
    else:
        return 'yellow'

def recognize_plate(image):
    # 定位车牌位置
    plate_rect = plate_detection(image)
    
    # 分割字符
    segments = segment_characters(image, plate_rect)

    # 识别字符
    characters = recognize_characters(segments)
    print(characters)
    print('\n')    
    # 识别车牌颜色
    color = recognize_color(image)
    
    # 将字符和车牌颜色组合起来得到最终的车牌号码
    plate_number = ''.join(characters) + ' ' + color
    
    return plate_number

# 读入图像
image = cv2.imread(r"C:\Users\m0815\Downloads\archive\images\Cars1.png")

# 显示OpenCV版本
print('OpenCV version:', cv2.__version__)

# 车牌识别
plate_number = recognize_plate(image)

# 显示识别结果
print('车牌号码为：', plate_number)


OpenCV version: 4.7.0
['', '', 're', '«\n«', '', '', '', '']


车牌号码为： re«
« blue


In [9]:
import pytesseract
import cv2
img=cv2.imread(r"C:\Users\m0815\Downloads\archive\images\Cars1.png")
characters=[]
# 设置识别语言为英文
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
custom_config = r'-l eng --oem 3 --psm 6'
character = pytesseract.image_to_string(img)
# characters.append(character)
print(character)